In [1]:
import torch
from transformers import EncodecModel, AutoProcessor, EncodecConfig

# Subclass EncodecModel to modify the forward method
class CustomEncodecModel(EncodecModel):
    def forward(self, input_values: torch.Tensor,
                padding_mask=None, bandwidth=None, audio_codes=None,
                audio_scales=None, return_dict=True):
        # Perform encoding only (no decoding)
        audio_codes, audio_scales = self.encode(input_values, padding_mask, bandwidth, False)
        return audio_codes  # Return only the encoded audio codes

def convert_encodec_to_onnx(model_name="facebook/encodec_24khz", output_path="../public/encodec_24khz.onnx"):
    # Load the custom Encodec model
    model = CustomEncodecModel.from_pretrained(model_name)
    model.eval()


    # Create dummy inputs with dynamic axes
    dummy_input = torch.randn(1, 1, 24000, dtype=torch.float32)  # 1 second of audio (for batch size 1)
    dummy_padding_mask = torch.zeros(1, 24000, dtype=torch.bool)  # Padding mask for sequence length

    print('Preparing to export model to ONNX') 

    # Export the model with dynamic batch size and sequence length
    torch.onnx.export(
        model,
        (dummy_input, dummy_padding_mask),
        output_path,
        export_params=True,
        opset_version=16,
        do_constant_folding=True,  # Apply constant folding optimization
        input_names=["input_values", "padding_mask",'bandwidth'],
        output_names=["audio_codes"],  # Only output the encoded audio codes
        dynamic_axes={
            "input_values": {0: "batch_size", 2: "sequence_length"},  # Dynamic axes for input
            "padding_mask": {0: "batch_size", 1: "sequence_length"},  # Dynamic axes for padding mask
            "audio_codes": {0: "batch_size"}  # Dynamic batch size for output
        }
    )
    print(f"ONNX model saved to {output_path}")

# Usage
convert_encodec_to_onnx()


c:\Users\User\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\miniconda3\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\User\miniconda3\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWar

Preparing to export model to ONNX


c:\Users\User\miniconda3\lib\site-packages\transformers\models\encodec\modeling_encodec.py:626: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if channels < 1 or channels > 2:
c:\Users\User\miniconda3\lib\site-packages\transformers\models\encodec\modeling_encodec.py:643: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_length % stride) - step != 0:
c:\Users\User\miniconda3\lib\site-packages\transformers\models\encodec\modeling_encodec.py:148: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't re

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX model saved to ../public/encodec_24khz.onnx
